In [ ]:
import numpy as np
import dolfinx
from mpi4py import MPI
from petsc4py import PETSc
import ufl
from dolfinx.mesh import create_rectangle, create_mesh
from dolfinx.fem import Function, functionspace, Constant, form
from dolfinx.fem.petsc import LinearProblem
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import pyvista as pv

from mpi4py import MPI

gii = nib.load("data/fsaverage.L.inflated.164k_fs_LR.surf.gii")

coords = None
tris   = None
for arr in gii.darrays:
    if arr.intent == 1008:
        coords = arr.data.copy()
    elif arr.intent == 1009:
        tris = arr.data.copy().astype(np.int64)

if coords is None or tris is None:
    raise RuntimeError("Missing POINTSET or TRIANGLE data in GIFTI.")

coords = coords.astype(np.float64)
tris   = tris.astype(np.int64)

from basix.ufl import element as BasixElement

domain = BasixElement("Lagrange", "triangle", 1, shape=(3,))

from dolfinx.mesh import create_mesh

mesh = create_mesh(MPI.COMM_WORLD, tris, coords, domain)
print(f"Loaded mesh with {mesh.geometry.x.shape[0]} vertices and {len(tris)} triangles.")

In [ ]:
new_coords = mesh.geometry.x.copy()
face_connectivity = mesh.topology.connectivity(2, 0).array.reshape(-1, 3)

n_faces = tris.shape[0]
faces = np.hstack([np.full((n_faces, 1), 3, dtype=np.int64), tris]).reshape(-1)
pv_mesh_original = pv.PolyData(coords, faces)

n_faces_new = face_connectivity.shape[0]
faces_new = np.hstack([np.full((n_faces_new, 1), 3, dtype=np.int64), face_connectivity]).reshape(-1)
pv_mesh_new = pv.PolyData(new_coords, faces_new)

plotter = pv.Plotter(shape=(1, 2), window_size=(1200, 600))
plotter.subplot(0, 0)
plotter.add_text("Original PV Mesh", font_size=12)
plotter.add_mesh(pv_mesh_original, color="lightgray", style="wireframe", line_width=0.5)
plotter.reset_camera()

plotter.subplot(0, 1)
plotter.add_text("DOLFINx-derived Mesh", font_size=12)
plotter.add_mesh(pv_mesh_new, color="lightgray", style="wireframe", line_width=0.5)
plotter.reset_camera()

plotter.show()

In [ ]:


gamma_s = 116
r_s = 30
dt = 10e-5
T = 0.01
num_time_steps = int(T / dt)


V = functionspace(mesh, ("CG", 1))

t = Constant(mesh, 0.0)


phi_n = Function(V)
phi_np1 = Function(V)
phi_nm1 = Function(V)
u_ref_dofs = np.zeros((num_time_steps, V.dofmap.index_map.size_local), dtype=np.float64)
Qs_dofs = np.zeros((num_time_steps, V.dofmap.index_map.size_local), dtype=np.float64)
t_star = np.linspace(0, T, num_time_steps)

Q0 = 0.05
alpha = 1.0
Nf = 17
Ns = 20
v_const = 1.0

freqs = np.arange(1, 85, 5)
freqs = freqs[:Nf]
rng = np.random.default_rng(42)
r0_sources = rng.uniform(-1.0, 1.0, size=(Ns, 3))
v_sources = rng.uniform(-1.0, 1.0, size=(Ns, 3))

def phase_func(r, r0, vs, time):
    r_s_of_t = r0 + vs * time
    diff = r.T - r_s_of_t
    dist = np.linalg.norm(diff, axis=1)
    return dist / v_const

def pink_noise_signal(x, time):
    out = np.zeros(x.shape[1], dtype=np.float64)
    for f in freqs:
        f_term = f**(-alpha)
        for s in range(Ns):
            r0_s = r0_sources[s]
            v_s = v_sources[s]
            Pi_vals = phase_func(x, r0_s, v_s, time)
            out += f_term * np.sin(2 * np.pi * f * (time + Pi_vals))
    norm_factor = Q0 / (len(freqs) * Ns)
    return norm_factor * out

def Q_callable(x):
    return pink_noise_signal(x, t.value)

Q_function = Function(V)


u_ufl = ufl.TrialFunction(V)

v_ufl = ufl.TestFunction(V)

A_form = (
    (1 / (gamma_s**2 * dt**2) + 1/(gamma_s * dt) + 1) * ufl.inner(u_ufl, v_ufl)
    + r_s**2 * ufl.dot(ufl.grad(u_ufl), ufl.grad(v_ufl))
)*ufl.dx

t.value = 0.0
Q_function.interpolate(lambda x: pink_noise_signal(x, t.value))
phi_n.x.array[:]   = Q_function.x.array
phi_nm1.x.array[:] = Q_function.x.array

for n in range(num_time_steps):

    t.value = (n + 1) * dt
    Q_function.interpolate(lambda x: pink_noise_signal(x, t.value))




    L_form = (
            (Q_function
             + (2 / (gamma_s**2 * dt**2)) * phi_n
            + (-1 / (gamma_s**2 * dt**2) + 1 /(gamma_s * dt))  * phi_nm1
             ) * v_ufl
        ) * ufl.dx
    problem = LinearProblem(A_form, L_form)
    phi_np1.x.array[:] = problem.solve().x.array

    u_ref_dofs[n, :] = phi_np1.x.array
    Qs_dofs[n, :] = Q_function.x.array

    phi_nm1.x.array[:] = phi_n.x.array
    phi_n.x.array[:] = phi_np1.x.array

    if n % 100 == 0:
        print(f"Step {n}, t={t.value:.3f}, max(phi)={phi_np1.x.array.max()}, min(phi)={phi_np1.x.array.min()}")

coords_3d = mesh.geometry.x
coords_2d = coords_3d[:, :2]



In [3]:
coords_3d = mesh.geometry.x
tris      = mesh.topology.connectivity(2, 0).array.reshape(-1, 3)

data_dict = {
    "mesh_coordinates": coords_3d,
    "connectivity":      tris,
    "t_star":            t_star,
    "phi_e":             u_ref_dofs,
    "Qs":                Qs_dofs
}

output_path = "NN_data/Reference Solution on Left Hemisphere: t=0.01s - 164k Mesh,dt=10e5.npy"
np.save(output_path, data_dict, allow_pickle=True)

In [ ]:
coords_3d

In [ ]:


coords = mesh.geometry.x.copy()
tris = mesh.topology.connectivity(2, 0).array.reshape(-1, 3)
faces = np.hstack([np.full((tris.shape[0], 1), 3, dtype=np.int64), tris]).reshape(-1)
pv_mesh = pv.PolyData(coords, faces)
pv_mesh["phi"] = phi_np1.x.array


p = pv.Plotter(window_size=(1000, 600))
p.add_mesh(pv_mesh, scalars="phi", cmap="viridis", show_edges=False, show_scalar_bar=False)


p.hide_axes()


p.add_scalar_bar(
    title="Phi Values",
    vertical=True,
    position_x=0.75,
    position_y=0.05,
    height=0.80,
    width=0.08,
    title_font_size=14,
    label_font_size=12,
    bold=True,
    above_label = '       '
)
p.add_text("Reference Solution on Left Hemisphere: t=0.01s - 164k Mesh dt=10e5",
            position="upper_edge",
            font_size=10,
            color="black")

p.show()

In [ ]:
 phi.x.array.shape